In [ ]:
import numpy as np
from Pyfhel.util import ENCODING_t
from Pyfhel import PyCtxt, Pyfhel, PyPtxt
import requests
import time

In [ ]:
ip_address = 'localhost'

In [ ]:
pyfhel = Pyfhel()
pyfhel.contextGen(p=65537, m=8192, base=2, sec=128, intDigits=64, fracDigits = 32)
pyfhel.keyGen()

In [ ]:
pyfhel.relinKeyGen(32,5)

In [ ]:
sepal_length_ctxt,sepal_width_ctxt,petal_length_ctxt,petal_width_ctxt = [pyfhel.encryptFrac(inp) for inp in test_input]

sepal_length_bytes = sepal_length_ctxt.to_bytes()
sepal_width_bytes = sepal_width_ctxt.to_bytes()
petal_length_bytes = petal_length_ctxt.to_bytes()
petal_width_bytes = petal_width_ctxt.to_bytes()


In [ ]:
ctxt = pyfhel.to_bytes_context()
pub = pyfhel.to_bytes_publicKey()
relin = pyfhel.to_bytes_relinKey()

In [ ]:
t = time.time()
url = f'http://{ip_address}:8080/register_key'
files = [('context',ctxt),('public_key',pub),('relin_key',relin)]

resp = requests.post(url=url,files=files)
print(resp.ok,'time taken',time.time()-t,'s')

In [ ]:
session_id = resp.json()['unique_id']

In [ ]:
url = f'http://{ip_address}:8080/predict_encrypted/iris/{session_id}'
files = [
    ('sepal_length',sepal_length_bytes),('sepal_width',sepal_width_bytes),
    ('petal_length',petal_length_bytes),('petal_width',petal_width_bytes)
]

resp_prediction = requests.post(url=url,files=files)
print(resp_prediction.ok)

In [ ]:
res_bytes = resp_prediction.content

In [ ]:
ans_ctext = PyCtxt(serialized=res_bytes,encoding = ENCODING_t.FRACTIONAL)

In [ ]:
ans_ctext

In [ ]:
pyfhel.decryptFrac(ans_ctext)

In [ ]:
url = f'http://{ip_address}:8080/predict_encrypted_perceptron/iris/{session_id}'
files = [
    ('sepal_length',sepal_length_bytes),('sepal_width',sepal_width_bytes),
    ('petal_length',petal_length_bytes),('petal_width',petal_width_bytes)
]

# This throws an error because linearize is not supported on server thread
resp_prediction = requests.post(url=url,files=files)
print(resp_prediction.ok)